### 前言

#### 结构介绍

config文件夹中应当由一个config_template.json文件用于储存模板。

In [ ]:
# ! git clone https://github.com/bwnotfound/Bert-VITS2.git
# ! cd Bert-VITS2

### Step 1
安装环境

In [ ]:

! pip install -r requirements.txt
%cd monotonic_align
! mkdir monotonic_align
%run setup.py build_ext --inplace
%cd ..

### Step 2

数据集预处理

请把数据集按照下面的格式放到dataset_raw文件夹下

```
dataset_raw
├───Character_name_1
├   ├───xxx.wav
├   ├───...
├   └───zzz.wav
├───Character_name_2
├   ├───xxx.wav
├   ├───...
├   └───zzz.wav
├───...
├
└───Character_name_n
    ├───xxx.wav
    ├───...
    └───zzz.wav
```

然后理解并按需运行下面的代码。

#### STEP 2.1

音频预处理

运行resample.py后会将dataset_raw文件夹下的音频文件转换成对应采样率的wav文件，并保存到dataset文件夹下。（和so-vits-svc的resample用法一样）

默认是响度匹配的。建议用其他软件处理完后直接放到dataset文件夹中

PS: 如果需要其他采样率，请加上类似```--sr 44100```的命令

In [ ]:
%run resample.py

#### STEP 2.2

语音转文字

该步骤适用于哪些没有文字标注的数据集，将使用whisper对齐文字。如果你有对应的文字标注，可以跳过这一步。

有哪些模型可以看annotate_wav.py源代码注释部分，使用例如```-o small```来选择模型大小。

语言选择上，一共有三种选项```C CJ CJE```，分别表示接受识别哪些语言，例如```CJE```就接受识别中文、日文、英文，然后排除其他的识别结果。默认```CJE```

默认是覆盖已有的annotation文件，如果不希望覆盖可以使用```-o False```

PS: 如果要跳过这一步，请把你的标注文件放到annotations文件夹下，文件名格式为```anno_{speaker_name}.txt```，例如```anno_niko.txt```

生成的文字标注格式为```{音频相对路径}|{说话人}|{语言类型}|{文字}```，例如```./dataset/niko/0001.wav|niko|EN|niko, hello~```


In [ ]:
# 这一步可能会在下载whisper模型出现过慢的问题，可以先把whiper模型下好放到download文件夹下，这样会更快一些
%run annotate_wav.py

#### STEP 2.3

整合annotation，生成训练集和测试集文件

这一步会将config_template.json转换为config.json

In [ ]:
%run preprocess_text.py

#### STEP 2.4

为对应音频文件生成bert.pt文件

bert/chinese-roberta-wwm-ext-large文件夹中缺少了pytorch_model.bin文件，需要到hugging face自行下载。

In [ ]:
%run spec_gen.py

#### STEP 3

训练

In [ ]:
# 这一步暂未详细测试，可能会有bug

! python train_ms.py

#### STEP 4

推理

In [ ]:
# 这一步暂未详细测试，可能会有bug

! python inference_webui.py